In [1]:
import clickhouse_connect
from embedding import E5LargeEmbeddingFunction
from similarity import bm25_ensemble_with_crossenc_answer, prep_query, bm25_fewshot_with_cross_encoder
from pprint import pprint
import requests
import json
import pandas as pd

/home/veidlink/HAKATONS/cb-purple-hack/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
pip freeze | findstr /v "@" > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
few_shot_data = pd.read_csv('data/finetune_df_final.csv', index_col=0)
few_shot_data

,doc_id,generated_question,generated_answer,is_markdown,context_batch
0,572.0,Какое полное и сокращенное фирменное наименова...,Полное фирменное наименование: Банк развития и...,0.0,Банковский сектор Информация о кредитных ор...
1,572.0,Какие данные представлены в отчете о финансовы...,В отчете о финансовых результатах формы 102 за...,0.0,на 1 февраля на 1 марта ...
2,572.0,Какие документы содержат информацию об обязате...,Информация об обязательных нормативах и других...,0.0,134 2015 год 2014 год 2013...
3,572.0,Какие документы включены в годовую (промежуточ...,В годовую (промежуточную) отчётность включаютс...,0.0,октября на 1 ноября ...
4,312.0,Какие основные направления развития финансовых...,В тексте описаны основные направления развития...,0.0,ОснОвные направления \nразвития финансОвых \...
...,...,...,...,...,...
348,1629.0,Какова доля плохих кредитов в процентах и как ...,"Доля плохих кредитов составляет 82.8%, а соотн...",0.0,.1\n6\n\n0\n6\n\n.1\n6\n\n0\n9\n\n.1\n6\n\n1\n...
349,1629.0,Какие данные представлены на графиках по росси...,На графиках представлены данные о торговых объ...,0.0,.1\n7\n\n0\n3\n\n.1\n8\n\n0\n6\n\n.1\n8\n\n0\n...
350,1629.0,Каковы торговые объемы на Московской бирже в т...,Торговые объемы на Московской бирже составляют...,0.0,8\n\n6\n.1\n\n8\n\n9\n.1\n\n8\n\nTrading volum...
351,1629.0,Что представляют собой данные '8% 11% 11% 12%'...,Эти данные представляют собой процентные ставк...,0.0,联邦公债券，万亿卢布 企业债券，万亿卢布\n\n8% 11% 11% 12%\n2% 4% ...


**Inference к модели, захосченной через ngrok & LM Studio**

In [3]:
client = clickhouse_connect.get_client(host='y1jzidyt9q.us-east-2.aws.clickhouse.cloud', port=8443, username='default', password='_lQ_JWXYQD3ym')
emb_func = E5LargeEmbeddingFunction()

In [4]:
def message_llm(system_prompt, user_input, temperature=0.6):
    '''Function to message (test) our LLM'''

    try:
        url = "https://live-relaxed-oryx.ngrok-free.app/v1/chat/completions"

        data = {
          "messages": [
            { "role": "system", "content": system_prompt},
            { "role": "user", "content": user_input}
          ],
          "temperature": 0.6,
          # "max_tokens": -1,
          # "stream": False
        }

        headers = {
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            return eval(response.text.replace("\n  ", ""))['choices'][0]['message']['content']
        else:
            print("Ошибка запроса:\n", response.status_code, response.text)
            return ""
            
    except Exception as e:
        
        print(f"Error {e}!")
        
rag_prompt = """
### CONTEXT ###
1) ИСТОЧНИК 1: 
- {type_of_source1}
- {link_to_source1}
- {batch_1}

2) ИСТОЧНИК 2: 
- {type_of_source2}
- {link_to_source2}
- {batch_2}

3) ИСТОЧНИК 3:
- {type_of_source3}
- {link_to_source3}
- {batch_3}

### INSTRUCTION ###
На основании приведенных данных категории «ТЕКСТ» дай исчерпывающий ответ на
приведенный ниже вопрос:
- {QUESTION}
"""

system_prompt = """
### ROLE ###
Ты - дружелюбный ассистент Центрального банка Российской Федерации. 
Исчерпывающе отвечай клиентам на возникшие вопросы. Если из контекста 
невозможно дать ответ на вопрос, пиши «Я не могу ответить на этот вопрос»."""

question = "Чем занимается ЦБ РФ?"
query = prep_query(question)

result = bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=15, cr_enc_n_results=3, limit_knn=100, knn_docs_window=1)

type_of_source1, type_of_source2, type_of_source3  = '', '', ''
link_to_source1, link_to_source2, link_to_source3 = '', '', ''
batch_1, batch_2, batch_3 = '', '', ''


for idx, batch in enumerate(start=1, iterable=result):
    if idx==1:
        if batch[5]:
            type_of_source1 = 'ТАБЛИЦА'
        else:
            type_of_source1 = 'ТЕКСТ'
        link_to_source1 = batch[2]
        batch_1 = batch[3]

    if idx==2:
        if batch[5]:
            type_of_source2 = 'ТАБЛИЦА'
        else:
            type_of_source2 = 'ТЕКСТ'
        link_to_source2 = batch[2]
        batch_2 = batch[3]

    if idx==3:
        if batch[5]:
            type_of_source3 = 'ТАБЛИЦА'
        else:
            type_of_source3 = 'ТЕКСТ'
        link_to_source3 = batch[2]
        batch_3 = batch[3]


formatted_rag_prompt = rag_prompt.format(
    type_of_source1=type_of_source1,
    link_to_source1=link_to_source1,
    batch_1=batch_1,
    type_of_source2=type_of_source2,
    link_to_source2=link_to_source2,
    batch_2=batch_2,
    type_of_source3=type_of_source3,
    link_to_source3=link_to_source3,
    batch_3=batch_3,
    QUESTION=question
)

answer = message_llm(system_prompt, formatted_rag_prompt)

print(f"\n{answer}")
print(f'\nИсточники: \n{link_to_source1}, \n{link_to_source2}, \n{link_to_source3}')

In [ ]:
bm25_fewshot_with_cross_encoder(bm25_n_results=100, cr_enc_n_results=3, df=few_shot_data, query=query)

[{'question': 'Какие функции будет осуществлять Банк России в рамках проведения операций денежно-кредитной политики?',
  'answer': 'Банк России будет осуществлять взаимодействие с кредитными организациями в рамках проведения операций денежно-кредитной политики.',
  'context': '•\u2002 направлять предписания, уведомления \nи  запросы Банка России участникам \nфинансового рынка и получать ответы;\n\n•\u2002 направлять организационно-распоря-\nдительные документы, в том числе ре-\nгламенты и порядки;\n\n•\u2002 получать отчетность в  электронном \nвиде от участников финансового рын-\nка, а также нерегламентированную ин-\nформацию (анкетирование, опросы) \nи статистические данные;\n\n•\u2002 осуществлять взаимодействие Бан-\nка России и  кредитных организаций \nв рамках проведения операций денеж-\nно-кредитной политики.\n\nС  целью обеспечения такого электрон-\nного взаимодействия будут разработаны \nнормативные (правовые) акты и  созданы \nличные кабинеты для участников финансо-\nвого рын

In [34]:
print(few_shot_examples)

[(506, 0, 'https://cbr.ru//about_br/', ' О Банке России Навигация по разделу: Правовой статус и функции История Национальный финансовый совет Совет директоров Банка России Организационная структура Территориальные учреждения Международное сотрудничество Отношения с инвесторами Противодействие коррупции Экспертные советы Пресс-служба Издания Банка России Мероприятия Университет Банка России Карьера в Банке России Внутренний аудит Закупки Банка России Фирменный стиль Миссия Банка России — обеспечение финансовой и ценовой стабильности, содействие развитию конкурентоспособного финансового рынка. Банк России обладает особым конституционно-правовым статусом, установленным cтатьей 75 Конституции Российской Федерации. В ней определено исключительное право Банка России на осуществление денежной эмиссии и в качестве основной функции — защита и обеспечение устойчивости рубля. Банк России осуществляет свою деятельность независимо от других федеральных органов государственной власти, органов власти

In [22]:
pprint(formatted_rag_prompt)

('\n'
 '### CONTEXT ###\n'
 '1) ИСТОЧНИК 1: \n'
 '- ТЕКСТ\n'
 '- https://cbr.ru//protection_rights/\n'
 '-  по телефонам контактного центра или в чате мобильного приложения «ЦБ '
 'онлайн». Операторы чата круглосуточно готовы ответить на вопросы о '
 'финансовых продуктах и услугах, порекомендовать порядок действий в сложной '
 'ситуации, проверить информацию о финансовой организации и многое другое. '
 'Если вам нужно отправить обращение в Банк России, это можно сделать через '
 'Интернет-приемную. Банк России отвечает на обращения в среднем в течение '
 'трех дней, однако рассмотрение сложных случаев может требовать больше '
 'времени. Предельный срок ответа финансовый уполномоченный не смог разрешить '
 'конфликт. Как обратиться в Банк России Связаться со специалистами Банка '
 'России можно по телефонам контактного центра или в чате мобильного '
 'приложения «ЦБ онлайн». Операторы чата круглосуточно готовы ответить на '
 'вопросы о финансовых продуктах и услугах, порекомендовать по